![logo](https://img-prod-cms-rt-microsoft-com.akamaized.net/cms/api/am/imageFileData/RE1Mu3b?ver=5c31)

# **Fabric**
### Simulating streaming data for Realtime Analytics ⚡ using fabric Data Engineering notebook 
### "Flight Streamer" ✈️ (Python) 
This notebook is used to take a CSV file extract that contains the complete flight track of one commercial flight that travelled from Toronto, On Canada to Calgary, Ab Canada. Every line of the CSV (1260 rows) represents a plane sighting. i.e. the aircraft transponder signal was detected by an open-source plan tracker and they recorded important data about the aircraft such as heading, velocity, longitude and latitude. The granularity of the data is one reading every 10 seconds.

The notebook will stream one row at at time (roughly 1 row/sec) into the custom app endpoint of the Event Stream. This will then be sent to the KQL database. The cell Marked "0" is where you set your parameters. We've populated it with our endpoint address but do make sure you replace it with yours.

The data used here was sourced from: 

The OpenSky Network, http://www.opensky-network.org 

or by citing the paper

Bringing up OpenSky: A large-scale ADS-B sensor network for research
Matthias Schäfer, Martin Strohmeier, Vincent Lenders, Ivan Martinovic, Matthias Wilhelm
ACM/IEEE International Conference on Information Processing in Sensor Networks, April 2014

(ii) You agree to expunge any and all copies of the received data set(s) upon completion or termination of stated research and/or termination of data access or use. Completion of stated research shall allow for a reasonable period of time that You may need to retain the data set(s) in order to satisfy scientific reproducibility obligations.

### **0. Set my connection string**

In [ ]:
MyConnectionString = ''

SampleCsv = 'abfss://3582b164-c42f-4707-98ac-a85e3bf6a734@msit-onelake.dfs.fabric.microsoft.com/f8a13080-852c-40d5-8181-c2bba079a710/Files/Sample/FlightsSample.csv'

### **1. Install dependencies and Event Hub library**

In [ ]:
pip install azure-eventhub>=5.11.0

In [ ]:
import time
import os
import datetime
import json
from azure.eventhub import EventHubProducerClient, EventData

### **2. Create a Python script to send events to your event stream**

ref: https://learn.microsoft.com/azure/event-hubs/event-hubs-capture-python#create-a-python-script-to-send-events-to-your-event-hub

In [ ]:
df = spark.read.csv(path=SampleCsv,header=True)

producer = EventHubProducerClient.from_connection_string(conn_str=MyConnectionString)

z = df.count() 

# df.collect()[0:5]
# df.collect()[5:10]

x = 0
y = 1

for x in range(0, z): 
    b = producer.create_batch() 
    j = df.toJSON().collect()[x:y]
    # print(j)
    b.add(EventData(j))
    producer.send_batch(b) # send it!
    time.sleep(1)
    producer.close()
    x=y
    y=y+1
